In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip install fasttext
!pip install emoji

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
     |████████████████████████████████| 71kB 2.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3017098 sha256=7c595f67be7b1c455d07d8c6e9845fe8b267b8438204ba68787f3c4d7387f324
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext
     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-

In [0]:
import pandas as pd
import numpy as np
import fasttext
import re
import sys
import os
import nltk
nltk.download('punkt')
import csv
import datetime
from bs4 import BeautifulSoup
import itertools
import emoji
import re
import string
from bs4 import BeautifulSoup
usecols = (2,4)
df = pd.read_table("/content/task1_training.tsv",usecols=usecols)
def preprocess_tweet(twt):
    twt = BeautifulSoup(twt).get_text()
    twt = twt.lower()
    twt = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','',twt)
    twt = re.sub('@[^\s]+','', twt)   
    twt = re.sub(r'#([^\s]+)', r'\1', twt)   
    twt = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", twt).split())
    twt = re.sub('[\s]+', ' ', twt)    
    twt = twt.strip()  
    return twt 
    


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
train_d =""
with open('SE_train.txt', 'w') as txtoutfile:
        for data in df.iterrows():
          txtoutfile.write('__label__'+str(data[1][1])+"\t"+ preprocess_tweet(data[1][0])+'\n' )
          train_d=train_d+'__label__'+str(data[1][1])+"\t"+ preprocess_tweet(data[1][0])+'\n' 

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "https://t.co/oewqnWJA4k" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "https://t.co/29HB3YRChW" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "https://t.co/SrTqQTcHuT" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful S

In [0]:
usecols = (2,4)
df = pd.read_table("/content/task1_validation.tsv",usecols=usecols)
ev_data=""
cnt = 0
with open('SE_validation.txt', 'w') as txtoutfile:
        for data in df.iterrows():
          if data[1][1] == 0 and cnt>200:
            continue
          if data[1][1] == 0:
            cnt = cnt+1
          txtoutfile.write('__label__'+str(data[1][1])+"\t"+ preprocess_tweet(data[1][0])+'\n' )
          ev_data=ev_data+'__label__'+str(data[1][1])+"\t"+ preprocess_tweet(data[1][0])+'\n'
ev_data=""
cnt = 0
with open('SE_train.txt', 'w') as txtoutfile:
        for data in df.iterrows():
          if data[1][1] == 0 and cnt>200:
            continue
          if data[1][1] == 0:
            cnt = cnt+1
          txtoutfile.write('__label__'+str(data[1][1])+"\t"+ preprocess_tweet(data[1][0])+'\n' )
          ev_data=ev_data+'__label__'+str(data[1][1])+"\t"+ preprocess_tweet(data[1][0])+'\n'

def train(training_data_path,validation_data_path):
 
    hyper_params = {"lr": 0.0001,
                    "epoch":10,
                    "wordNgrams": 2,
                    "dim": 10}     

    model = fasttext.train_supervised(input=training_data_path, **hyper_params)
    print("hyperparameters used \n {}".format(hyper_params))

    model_acc_validation_set = model.test(validation_data_path)

    text =  "f1-score:" + str(model_acc_validation_set[1]) + '\n'
    print(text)

In [0]:
training_data_path = os.path.join("SE_train.txt")
validation_data_path = os.path.join("SE_validation.txt")
train("/content/SE_train.txt","/content/SE_validation.txt")

hyperparameters used 
 {'lr': 0.0001, 'epoch': 10, 'wordNgrams': 2, 'dim': 10}
f1-score:0.7838983050847458

